In [1]:
! pip install pandas openpyxl sqlalchemy pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.3 MB/s eta 0:00:00


In [2]:
# load excel to sql

from sqlalchemy import create_engine

# PostgreSQL connection details from the provided environment variables
project_name = 'magic-zoomcamp'
db_name = 'postgres'

user = 'postgres'
password = 'postgres'
host = '192.168.0.84'
port = '5432'

# Create connection string
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}"

# Create an engine instance
engine = create_engine(connection_string)

# Test the connection
with engine.connect() as connection:
    print("Connection to the PostgreSQL database established successfully.")


Connection to the PostgreSQL database established successfully.


In [72]:
import os
# Directory containing Excel files
excel_folder_path = '../enriched/'

# Get a list of all Excel files in the folder
excel_files = [os.path.join(excel_folder_path,f) for f in os.listdir(excel_folder_path) if f.endswith('.xlsx') or f.endswith('.xls')]

print("Excel files found:",excel_files)

Excel files found: ['../enriched/enriched_Statement_3_2024.xlsx', '../enriched/enriched_Statement_4_2023.xlsx', '../enriched/enriched_Statement_9_2023.xlsx', '../enriched/enriched_Statement_12_2023.xlsx', '../enriched/enriched_Statement_5_2024.xlsx', '../enriched/enriched_Statement_2_2023.xlsx', '../enriched/enriched_Statement_2_2024.xlsx', '../enriched/enriched_Statement_5_2023.xlsx', '../enriched/enriched_Statement_8_2023.xlsx', '../enriched/enriched_Statement_4_2024.xlsx', '../enriched/enriched_Statement_3_2023.xlsx', '../enriched/enriched_Statement_6_2023.xlsx', '../enriched/enriched_Statement_1_2024.xlsx', '../enriched/enriched_Statement_10_2023.xlsx', '../enriched/enriched_Statement_7_2023.xlsx', '../enriched/enriched_Statement_1_2023.xlsx', '../enriched/enriched_Statement_6_2024.xlsx', '../enriched/enriched_Statement_11_2023.xlsx']


In [73]:
excel_files

['../enriched/enriched_Statement_3_2024.xlsx',
 '../enriched/enriched_Statement_4_2023.xlsx',
 '../enriched/enriched_Statement_9_2023.xlsx',
 '../enriched/enriched_Statement_12_2023.xlsx',
 '../enriched/enriched_Statement_5_2024.xlsx',
 '../enriched/enriched_Statement_2_2023.xlsx',
 '../enriched/enriched_Statement_2_2024.xlsx',
 '../enriched/enriched_Statement_5_2023.xlsx',
 '../enriched/enriched_Statement_8_2023.xlsx',
 '../enriched/enriched_Statement_4_2024.xlsx',
 '../enriched/enriched_Statement_3_2023.xlsx',
 '../enriched/enriched_Statement_6_2023.xlsx',
 '../enriched/enriched_Statement_1_2024.xlsx',
 '../enriched/enriched_Statement_10_2023.xlsx',
 '../enriched/enriched_Statement_7_2023.xlsx',
 '../enriched/enriched_Statement_1_2023.xlsx',
 '../enriched/enriched_Statement_6_2024.xlsx',
 '../enriched/enriched_Statement_11_2023.xlsx']

In [74]:
import pandas as pd

# Replace 'your_excel_file.xlsx' with the path to your Excel file
# excel_file_path = '../enriched/enriched_Statement_1_2023.xlsx'
df = pd.concat([pd.read_excel(excel_file) for excel_file in excel_files])
# pd.read_excel(excel_files)

In [75]:
df.columns = ['transaction_date',  'description', 'category',
       'type', 'money_in', 'money_out', 'balance']

In [76]:
df['year'] = df['transaction_date'].dt.year
df['month'] = df['transaction_date'].dt.month
df['day'] = df['transaction_date'].dt.day


In [77]:
# Define the table name where you want to load the data
schema = 'statements_enriched'
table_name = 'lloyds_enriched'

# Load the DataFrame into the SQL table
df.to_sql(table_name, con=engine, schema=schema, if_exists='replace', index=False)

print(f"Data has been successfully loaded into the '{schema}.{table_name}' table.")


Data has been successfully loaded into the 'statements_enriched.lloyds_enriched' table.


In [78]:
query = "SELECT * FROM statements_staging.lloyds_staging"
pd.read_sql(query,engine).columns

Index(['transaction_date', 'year', 'month', 'day', 'description', 'tag',
       'type', 'money_in', 'money_out', 'balance'],
      dtype='object')

In [82]:
df.groupby("category")['money_in'].sum()

category
Bills                  0.00
Eating out             0.00
Entertainment          0.00
Groceries              0.00
Health & Beauty        0.00
Income             20780.48
Investment           384.55
Loan                3528.32
Other                900.47
Rent                 743.49
Savings              350.00
Shopping               4.49
Study                  0.00
Transfers           4906.57
Transport            228.05
Name: money_in, dtype: float64